In [1]:
from entities import *
from constants import TEAMS_DATA, STAT_EFFECTS, POSSESSIONS_STD_DEV

import pickle
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

In [2]:
with open(TEAMS_DATA, 'rb') as f:
    teams = pickle.load(f)

with open(STAT_EFFECTS, 'rb') as f:
    stats = pickle.load(f)

In [3]:
SCALED_STATS = ['2PA', '3PA', 'FTA', 'STL', 'BLK', 'TOV', 'PF', 'ORB', 'DRB']
UNSCALED_STATS = ['2P%', '3P%', 'FT%']
MODELED_STATS = ['BLK', 'STL', 'TOV', 'ORB', 'DRB', 'PF']

def get_starters(team: Team):
    starters = list(team.df['GS'].sort_values(ascending=False).iloc[:5].index)
    starters.sort()
    return starters


def get_random_players(team: Team):
    pct_played = team.df['MP'] / (team.games * 40)
    players = np.array(pct_played.index)
    weights = np.array(pct_played) / np.array(pct_played).sum()

    sampled_players = list(np.random.choice(players, size=5, replace=False, p=weights))
    sampled_players.sort()
    return sampled_players


def get_player_stats(player: str, team: Team):
    return team.df.loc[player]


def get_number_of_possessions(team_1: Team, team_2: Team):
    μ = np.mean([team_1.pace, team_2.pace])
    σ = POSSESSIONS_STD_DEV
    
    p = np.random.normal(loc=μ, scale=σ)
    j = np.random.uniform(low=-2, high=2, size=2)

    p1 = float(p + j[0])
    p2 = float(p + j[1])
    return p1, p2


def simulate_game(team_1: Team, team_2: Team):
    output = {}
    game_log = []
    
    possessions = get_number_of_possessions(team_1, team_2)

    for i, t in enumerate([team_1, team_2]):
        simulated_stats = {}
        for player in get_random_players(t):
            event_log = {}
            tm_score = 0
            opp_score  = 0
            player_stats = get_player_stats(player, t)[SCALED_STATS] * (possessions[i] / 100)
            shooting_pct = get_player_stats(player, t)[UNSCALED_STATS]

            _2p = 2.0 * player_stats['2PA'] * shooting_pct['2P%']
            _3p = 3.0 * player_stats['3PA'] * shooting_pct['3P%']
            _pts = float(_2p + _3p)

            event_log['Pts'] = _pts
            tm_score += _pts

            for event in ['BLK', 'STL', 'TOV', 'ORB', 'DRB', 'PF']:
                tm_score += player_stats[event] * stats[event]['Tm']
                opp_score  += player_stats[event] * stats[event]['Opp_Score']
                event_log[event] = float(player_stats[event])

            simulated_stats[player] = [float(tm_score), float(opp_score), event_log]
        game_log.append(simulated_stats)

    t1 = np.array(list(game_log[0].values()))
    t2 = np.array(list(game_log[1].values()))
    team_1_score = np.sum(t1[:, 0]) + np.sum(t2[:, 1])
    team_2_score = np.sum(t1[:, 1]) + np.sum(t2[:, 0])
    
    if team_1_score > team_2_score:
        output['Win'] = team_1.name
        output['Win Score'] = team_1_score
        output['Lose'] = team_2.name
        output['Lose Score'] = team_2_score

    elif team_1_score < team_2_score:
        output['Win'] = team_2.name
        output['Win Score'] = team_2_score
        output['Lose'] = team_1.name
        output['Lose Score'] = team_1_score

    elif np.nan in [team_1_score, team_2_score]:
        return simulate_game(team_1, team_2)
    
    else:
        return simulate_game(team_1, team_2)

    output['Game Log'] = game_log
    return output

In [4]:
az = teams['Arizona State']
al = teams['Alabama']
b = teams['Baylor']
de = teams['Denver']

In [6]:
%%timeit
simulate_game(b, de)

84.6 ms ± 26.9 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [7]:
simulate_game(b, de)


{'Win': 'Baylor',
 'Win Score': 92.21274395959708,
 'Lose': 'Denver',
 'Lose Score': 75.15572228073738,
 'Game Log': [{'Cameron Carr': [22.626839857901587,
    -1.9808575428568105,
    {'Pts': 19.38738347049729,
     'BLK': 1.5854233147788412,
     'STL': 0.758245933155098,
     'TOV': 2.3436692479339394,
     'ORB': 1.1718346239669697,
     'DRB': 5.4455844290229765,
     'PF': 2.205806350996649}],
   'Isaac Williams': [16.24244941043378,
    3.424103970955943,
    {'Pts': 14.428110410420611,
     'BLK': 0.2067943454059358,
     'STL': 1.3786289693729055,
     'TOV': 2.067943454059358,
     'ORB': 0.4825201392805169,
     'DRB': 2.757257938745811,
     'PF': 4.894132841273814}],
   'Michael Rataj': [12.213774344043918,
    0.6358127521441137,
    {'Pts': 9.051595292763217,
     'BLK': 0.8271773816237432,
     'STL': 1.6543547632474864,
     'TOV': 2.4815321448712298,
     'ORB': 4.9630642897424595,
     'DRB': 3.101915181089037,
     'PF': 2.826189387214456}],
   'Obi Agbim': [12.2397